In [1]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
%matplotlib inline

In [5]:
# Define the directory you want to list files from
directory = '/corral/utexas/UKB-Imaging-Genetics/temp_imaging_data/FID20205_ECG/'

# Initialize an empty list to store file paths
file_paths = []
patient_id = []
visit_no = []

# Use os.walk to get all file paths
for root, dirs, files in os.walk(directory):
    for file in files:
        ids1 = file.split('.')[0]
        ids = ids1.split('_')[0]
        visit = ids1.split('_')[2]
        patient_id.append(int(ids))
        file_paths.append(os.path.join(root, file))
        visit_no.append(int(visit))
        

# Create a DataFrame from the list of file paths
df_ecg = pd.DataFrame(file_paths, columns=['FilePath_ECG'])
df_ecg['IID'] = patient_id
df_ecg['visit'] = visit_no
df_ecg = df_ecg.loc[df_ecg['visit'] == 2]
df_ecg = df_ecg.drop(['visit'], axis = 1)

cad_cases = pd.read_csv('cad_case_control.csv')
cad_cases = cad_cases.rename(columns={'Patient EID':'IID'})

import pandas as pd

# Load and prepare data
df_ecg = pd.merge(df_ecg, cad_cases, on='IID', how='inner')
df_test = pd.read_csv('Final_Datasets/test_data_incidence.csv')
controls_additional = pd.read_csv('Final_Datasets/imbalanced_control_iid.csv')
controls_additional = controls_additional[['IID', 'CAD', 'FilePath_ECG']]

# Remove test cases and controls
test_iids = set(df_test['IID'])

# Get all cases directly from df_ecg, excluding test data
cases = df_ecg[(df_ecg['CAD'] == 1) & (~df_ecg['IID'].isin(test_iids))]

# Get all controls specifically from controls_additional, excluding test data
controls = controls_additional[~controls_additional['IID'].isin(test_iids)]

# Combine cases and controls into an imbalanced dataset
df_imbalanced_train = pd.concat([cases, controls], axis=0).reset_index(drop=True)

# Save the imbalanced dataset
df_imbalanced_train.to_csv('Final_Datasets/train_imbalanced_ecg_temp.csv', sep=',', index=False)

# Report dataset statistics
cases_count = len(cases)
controls_count = len(controls)
print(f"Number of cases: {cases_count}")
print(f"Number of controls: {controls_count}")
print(f"Imbalance ratio (controls to cases): {controls_count / cases_count:.2f}")

Number of cases: 4329
Number of controls: 26358
Imbalance ratio (controls to cases): 6.09


In [ ]:
import pandas as pd

# Load and prepare data
df_ecg = pd.merge(df_ecg, cad_cases, on='IID', how='inner')
df_test = pd.read_csv('Final_Datasets/test_data_incidence.csv')
controls_additional = pd.read_csv('Final_Datasets/imbalanced_control_iid.csv')
controls_additional = controls_additional[['IID', 'CAD', 'FilePath_ECG']]

# Remove test cases and controls
test_iids = set(df_test['IID'])

# Get all cases directly from df_ecg, excluding test data
cases = df_ecg[(df_ecg['CAD'] == 1) & (~df_ecg['IID'].isin(test_iids))]

# Get all controls specifically from controls_additional, excluding test data
controls = controls_additional[~controls_additional['IID'].isin(test_iids)]

# Combine cases and controls into an imbalanced dataset
df_imbalanced_train = pd.concat([cases, controls], axis=0).reset_index(drop=True)

# Save the imbalanced dataset
df_imbalanced_train.to_csv('Final_Datasets/train_imbalanced_ecg_temp.csv', sep=',', index=False)

# Report dataset statistics
cases_count = len(cases)
controls_count = len(controls)
print(f"Number of cases: {cases_count}")
print(f"Number of controls: {controls_count}")
print(f"Imbalance ratio (controls to cases): {controls_count / cases_count:.2f}")

In [6]:
df_ecg.head()

,FilePath_ECG,IID,CAD
0,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,4551956,0
1,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,2650691,0
2,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,2807784,0
3,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,5192069,0
4,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,5785208,0


In [7]:
df_imbalanced_train["CAD"].value_counts()

CAD
0    26358
1     4329
Name: count, dtype: int64

In [8]:
df_imbalanced_train.head()

,FilePath_ECG,IID,CAD
0,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,4465837,1
1,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,4611362,1
2,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,4032904,1
3,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,5099267,1
4,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,3910439,1


In [9]:
df_test.head()

,FilePath_ECG,IID,FilePath,CAD,FilePath_liver,FilePath_pancreas,FilePath_dxa,FilePath_bf,PRS,age,...,bp_sys,pulse_rate,QRSDuration,QTInterval,QTCInterval,RRInterval,PPInterval,MeanQAmplitude,MeanRAmplitude,MeanTAmplitude
0,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,4890586,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,0,/corral-repl/utexas/UKB-Imaging-Genetics/Imagi...,/corral-repl/utexas/UKB-Imaging-Genetics/Imagi...,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,/scratch/07880/devansh/CAD/FID20213_blood_flow...,0.256425,54.0,...,126.0,48.0,90.0,466.0,416.0,1238.0,1250.0,67.833333,874.750000,110.416667
1,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,2201189,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,0,/corral-repl/utexas/UKB-Imaging-Genetics/Imagi...,/corral-repl/utexas/UKB-Imaging-Genetics/Imagi...,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,/scratch/07880/devansh/CAD/FID20213_blood_flow...,0.143094,71.0,...,152.0,60.0,88.0,422.0,424.0,988.0,980.0,120.833333,801.083333,287.416667
2,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,3646967,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,1,/corral-repl/utexas/UKB-Imaging-Genetics/Imagi...,/corral-repl/utexas/UKB-Imaging-Genetics/Imagi...,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,/scratch/07880/devansh/CAD/FID20213_blood_flow...,0.334331,72.0,...,137.0,74.0,82.0,422.0,428.0,968.0,965.0,329.000000,356.416667,202.916667
3,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,2032754,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,0,/corral-repl/utexas/UKB-Imaging-Genetics/Imagi...,/corral-repl/utexas/UKB-Imaging-Genetics/Imagi...,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,/scratch/07880/devansh/CAD/FID20213_blood_flow...,-0.017397,58.0,...,117.0,88.0,90.0,406.0,425.0,904.0,905.0,230.666667,472.416667,140.833333
4,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,2434644,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,1,/corral-repl/utexas/UKB-Imaging-Genetics/Imagi...,/corral-repl/utexas/UKB-Imaging-Genetics/Imagi...,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,/scratch/07880/devansh/CAD/FID20213_blood_flow...,0.173801,74.0,...,149.0,48.0,114.0,458.0,429.0,1140.0,1130.0,270.500000,1020.833333,475.833333


In [11]:
# Function to extract ECG features from a given XML file
def extract_ecg_features(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Extract Intervals and Durations
        pq_interval = root.findtext(".//PQInterval")
        p_duration = root.findtext(".//PDuration")
        qrs_duration = root.findtext(".//QRSDuration")
        qt_interval = root.findtext(".//QTInterval")
        qtc_interval = root.findtext(".//QTCInterval")
        rr_interval = root.findtext(".//RRInterval")
        pp_interval = root.findtext(".//PPInterval")

        # Convert to numeric values
        pq_interval = float(pq_interval) if pq_interval else None
        p_duration = float(p_duration) if p_duration else None
        qrs_duration = float(qrs_duration) if qrs_duration else None
        qt_interval = float(qt_interval) if qt_interval else None
        qtc_interval = float(qtc_interval) if qtc_interval else None
        rr_interval = float(rr_interval) if rr_interval else None
        pp_interval = float(pp_interval) if pp_interval else None

        # Extract Amplitudes from the MeasurementTable
        p_amplitudes = []
        q_amplitudes = []
        r_amplitudes = []
        t_amplitudes = []

        # Extract amplitude values from the 'MeasurementTable' for each lead
        measurement_table = root.find(".//MeasurementTable")
        if measurement_table is not None:
            p_wave = measurement_table.findtext("PAmplitude")
            q_wave = measurement_table.findtext("QAmplitude")
            r_wave = measurement_table.findtext("RAmplitude")
            t_wave = measurement_table.findtext("TAmplitude")

            # Convert to lists of floats
            p_amplitudes = [float(value) for value in p_wave.split(',') if value.strip()]
            q_amplitudes = [float(value) for value in q_wave.split(',') if value.strip()]
            r_amplitudes = [float(value) for value in r_wave.split(',') if value.strip()]
            t_amplitudes = [float(value) for value in t_wave.split(',') if value.strip()]

            # Calculate mean amplitudes
            mean_p_amplitude = np.mean(p_amplitudes) if p_amplitudes else None
            mean_q_amplitude = np.mean(q_amplitudes) if q_amplitudes else None
            mean_r_amplitude = np.mean(r_amplitudes) if r_amplitudes else None
            mean_t_amplitude = np.mean(t_amplitudes) if t_amplitudes else None
        else:
            mean_p_amplitude = mean_q_amplitude = mean_r_amplitude = mean_t_amplitude = None

        # Return extracted features
        return {
            'PQInterval': pq_interval,
            'PDuration': p_duration,
            'QRSDuration': qrs_duration,
            'QTInterval': qt_interval,
            'QTCInterval': qtc_interval,
            'RRInterval': rr_interval,
            'PPInterval': pp_interval,
            'MeanPAmplitude': mean_p_amplitude,
            'MeanQAmplitude': mean_q_amplitude,
            'MeanRAmplitude': mean_r_amplitude,
            'MeanTAmplitude': mean_t_amplitude
        }

    except ET.ParseError:
        # Handle parsing errors if any XML files are not well-formed
        print(f"Error parsing XML file: {file_path}")
        return {
            'PQInterval': None,
            'PDuration': None,
            'QRSDuration': None,
            'QTInterval': None,
            'QTCInterval': None,
            'RRInterval': None,
            'PPInterval': None,
            'MeanPAmplitude': None,
            'MeanQAmplitude': None,
            'MeanRAmplitude': None,
            'MeanTAmplitude': None
        }


# Initialize new columns in the DataFrame
for feature in ['PQInterval', 'PDuration', 'QRSDuration', 'QTInterval', 'QTCInterval', 'RRInterval', 'PPInterval',
                'MeanPAmplitude', 'MeanQAmplitude', 'MeanRAmplitude', 'MeanTAmplitude']:
    df_imbalanced_train[feature] = None

# Iterate over DataFrame rows and update with extracted features
for index, row in df_imbalanced_train.iterrows():
    features = extract_ecg_features(row['FilePath_ECG'])
    for feature_name, feature_value in features.items():
        df_imbalanced_train.at[index, feature_name] = feature_value

In [12]:
df_imbalanced_train.head()

,FilePath_ECG,IID,CAD,PQInterval,PDuration,QRSDuration,QTInterval,QTCInterval,RRInterval,PPInterval,MeanPAmplitude,MeanQAmplitude,MeanRAmplitude,MeanTAmplitude
0,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,4465837,1,152.0,96.0,78.0,376.0,431.0,754.0,755.0,55.416667,302.5,339.583333,185.4
1,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,4611362,1,162.0,126.0,78.0,404.0,423.0,908.0,905.0,36.833333,336.5,562.0,125.416667
2,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,4032904,1,138.0,96.0,96.0,398.0,429.0,862.0,855.0,33.0,45.333333,574.833333,104.25
3,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,5099267,1,208.0,96.0,132.0,440.0,471.0,868.0,865.0,44.5,854.0,319.555556,175.75
4,/corral/utexas/UKB-Imaging-Genetics/temp_imagi...,3910439,1,144.0,100.0,78.0,354.0,430.0,676.0,670.0,61.916667,104.285714,515.666667,127.583333


In [13]:
df_imbalanced_train.isnull().sum()

FilePath_ECG         0
IID                  0
CAD                  0
PQInterval        1885
PDuration         1849
QRSDuration          5
QTInterval           5
QTCInterval          5
RRInterval           5
PPInterval           5
MeanPAmplitude     798
MeanQAmplitude       8
MeanRAmplitude       6
MeanTAmplitude       8
dtype: int64

In [15]:
df_imbalanced_train = df_imbalanced_train.drop(['PQInterval', 'PDuration', 'MeanPAmplitude'], axis = 1)
df_imbalanced_train.isnull().sum()

FilePath_ECG      0
IID               0
CAD               0
QRSDuration       5
QTInterval        5
QTCInterval       5
RRInterval        5
PPInterval        5
MeanQAmplitude    8
MeanRAmplitude    6
MeanTAmplitude    8
dtype: int64

In [16]:
df_imbalanced_train = df_imbalanced_train.dropna()

In [17]:
df_imbalanced_train.isnull().sum()

FilePath_ECG      0
IID               0
CAD               0
QRSDuration       0
QTInterval        0
QTCInterval       0
RRInterval        0
PPInterval        0
MeanQAmplitude    0
MeanRAmplitude    0
MeanTAmplitude    0
dtype: int64

In [20]:
df_imbalanced_train.to_csv('Final_Datasets/train_imbalanced_ecg.csv', index = False)

In [3]:
df = pd.read_csv('Final_Datasets/train_imbalanced_ecg.csv')

In [5]:
df.describe(include=[np.number])

,IID,CAD,QRSDuration,QTInterval,QTCInterval,RRInterval,PPInterval,MeanQAmplitude,MeanRAmplitude,MeanTAmplitude
count,3.067600e+04,30676.000000,30676.000000,30676.000000,30676.000000,30676.000000,30676.000000,30676.000000,30676.000000,30676.000000
mean,3.287576e+06,0.141022,88.741883,420.706024,422.782468,996.691029,989.135481,219.753949,642.534998,184.001535
std,1.330764e+06,0.348050,15.063731,32.128885,25.272794,166.154227,187.609947,219.786843,210.676648,96.552300
min,1.000026e+06,0.000000,34.000000,150.000000,203.000000,298.000000,0.000000,19.000000,76.916667,-3701.000000
25%,2.141681e+06,0.000000,80.000000,400.000000,406.000000,884.000000,880.000000,71.800000,496.979167,121.909091
50%,3.285220e+06,0.000000,86.000000,420.000000,422.000000,990.000000,980.000000,147.666667,621.750000,176.666667
75%,4.430202e+06,0.000000,94.000000,440.000000,438.000000,1100.000000,1090.000000,287.800000,765.020833,240.416667
max,6.024771e+06,1.000000,252.000000,742.000000,875.000000,2246.000000,3525.000000,6254.500000,6983.166667,669.500000
